# Hàm Valid_Matrix để thêm cái weight vào với nhau 

In [1]:
import numpy as np 
from numba import njit 
def is_linearly_independent(matrix):
    rank = np.linalg.matrix_rank(matrix)
    return rank == matrix.shape[0]


True


# Cách rút gọn search space để tìm bộ weight 

**Tạo ra toàn bộ search space cho weight của 1 feature**

In [6]:
size = 10

# generate all possible combinations of indices for the 1 and -1
combinations = [(i, j) for i in range(size) for j in range(size) if i != j]

# create the arrays using the combinations
arrays = []
for (i, j) in combinations:
  arr = [0] * size
  arr[i] = 1
  arr[j] = -1
  arrays.append(arr)

# print the resulting arrays
print(len(arrays))


30


**sinh ma trận co 3 giá trị 1 ,1 và -1**

In [3]:
import numpy as np
from itertools import combinations

# Generate all possible combinations of indices for the 1s and -1
indices = combinations(range(161), 3)

# Filter out combinations where the indices of the two 1s are the same
indices = [x for x in indices if x[0] != x[1]]

# Create an empty list to store the arrays
arrays = []

# Iterate over the combinations of indices
for i, j, k in indices:
  # Create a new array with 1s at the specified indices and -1 at the other index
  arr = np.zeros(161)
  arr[i] = 1
  arr[j] = 1
  arr[k] = -1
  # Append the array to the list
  arrays.append(arr)

# Print the resulting list of arrays
print(np.array(arrays).shape)


(682640, 161)


**Sau khi sử dụng hàm Valid_Matrix**

In [12]:
import itertools

def all_submatrices_and_check(matrix, num_rows):
    # Get the number of rows and columns in the matrix
    # Iterate over all possible combinations of rows
    for combination in itertools.combinations(range(30), num_rows):
        # Create a new submatrix using the selected rows
        submatrix = [matrix[row] for row in combination]
        # Check if the submatrix is linearly independent
        if is_linearly_independent(np.array(submatrix)):
            # Yield the submatrix
            yield np.array(submatrix)

# Example usage
all_submatris_check = []
matrix = arrays
for submatrix in all_submatrices_and_check(matrix, 5):
        all_submatris_check.append(submatrix)
        # print(submatrix)

print(len(all_submatris_check))
import itertools
matrix = arrays

def all_submatrices(matrix, num_rows):
    # Get the number of rows and columns in the matrix
    # Iterate over all possible combinations of rows
    for combination in itertools.combinations(range(30), num_rows):
        # Create a new submatrix using the selected rows
        submatrix = [matrix[row] for row in combination]
        # Check if the submatrix is linearly independent
        # Yield the submatrix
        yield np.array(submatrix)
all_submatris = []
for submatrix in all_submatrices(matrix, 5):
        all_submatris.append(submatrix)
print(len(all_submatris))
print('rate of linearly independent submatrices: ', len(all_submatris_check)/len(all_submatris))

41472
142506
rate of linearly independent submatrices:  0.29101932550208415


**Tính số trận thắng của từng ma trận**

In [49]:
%cd /Users/trancatkhanh/Downloads/GAME
from base.Splendor_v3.env import *
high_values_matrix = []
k = 0
for i in range(len(all_submatris)):
    k +=1
    perx = all_submatris[i].astype(np.float64)
    @njit
    def p0(state,per):
        actions = getValidActions(state)
        matran1 = per@state    
        matran1 = 1*(matran1 >= 0)
        if np.sum(matran1) == 5 and 0 in actions :
            action = 0
        return action,per
    win1, x = numba_main_2(p0, 100,perx,0)
    print('process : {}/{}'.format(k, len(all_submatris)), end='\r')
    high_values_matrix.append(win1)

/Users/trancatkhanh/Downloads/GAME


# Thử lại số trận thắng


In [ ]:
print(per1)

In [90]:
import numpy as np
from numba import njit 
#Khởi tạo weight 
perx = [np.zeros((60,getStateSize())),np.zeros((12,60))]
for i in range(0,12):
    for j in range(5):
        perx[0][i*5+j][6+j] = 1
        perx[0][i*5+j][11] = 0 
        perx[0][i*5+j][12+j] = 1
        perx[0][i*5+j][20+i*7+j] = -1
for i in range(0,12):
    for j in range(5):
        perx[1][i][i*5+j] = 1
for i in range(len(all_submatris)):
    if (all_submatris[i]== perx[0][0:5].astype(int)).all():
        print(i)


In [50]:
per1 = all_submatris[high_values_matrix.index(max(high_values_matrix))].astype(np.float64)


In [16]:
from base.Splendor_v3.env import *
import numpy as np
bias1 = [np.random.rand(getActionSize()),np.zeros(getActionSize())]
@njit()
def p_bias(state,per):
    actions = getValidActions(state)
    actions *= per[0]
    action = np.argmax(actions)
    if getReward(state) == 1:
        per[1] += per[0]
    if getReward(state) == 0:
        per[0] = np.random.rand(getActionSize())
    return action,per

win, bias = numba_main_2(p_bias, 50000,bias1,0)
# Bias tối ưu 
bias_greedy = (bias[1]/np.max(bias[1]))
print(bias_greedy)
# Check tỷ lệ thắng của bias tối ưu 
@njit()
def p_bias_optim(state,per):
    actions = getValidActions(state)
    actions *= bias_greedy
    action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(p_bias_optim, 100,bias1,0)
print(win)

[0.9288805  0.93778639 0.98036402 1.         0.89458567 0.89399649
 0.89721242 0.89451654 0.89911454 0.8968883  0.89599888 0.89894841
 0.90363768 0.89228768 0.89890005]
62


In [52]:
@njit
def p0(state,per):
    actions = getValidActions(state)
    matran1 = per@state    
    matran1 = 1*(matran1 >= 0)
    actions = np.where(actions == 1)[0]
    if np.sum(matran1) == 5 and 0 in actions :
        action = 0
    else:
        list_action2 = getValidActions(state)
        action = np.argmax(list_action2*bias_greedy)
    return action,per
win1, x = numba_main_2(p0, 1000,per1,0)
print('phần trăm trận thắng 1000 của action ',win1/1000)

phần trăm trận thắng 1000 của action  0.44
